In [1]:
# imports
import pandas as pd
import openai 
import ast
import numpy as np
from sklearn.metrics import precision_score, recall_score
import random

Data

In [2]:
interactions = pd.read_csv("MIND/behaviorsTEST.tsv",sep='\t',  header=None)
interactions.columns =['User', 'Time', 'ID', 'Impressions'] 
interactions = interactions.drop(['Time'], axis=1)
interactions.head()

,User,ID,Impressions
0,U13740,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
1,U91836,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,U73700,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...


In [4]:
#load the data with news articles
news = pd.read_csv("embeddings/news_emb_TEST.csv") #document with news content
news.head()

,ID,Category,SubCategory,Content,Content_emb
0,N55189,tv,tvnews,"'Wheel Of Fortune' Guest Delivers Hilarious, O...","[-0.020967688411474228, -0.020634232088923454,..."
1,N42782,sports,baseball_mlb,Three takeaways from Yankees' ALCS Game 5 vict...,"[-0.008319429121911526, 0.0013667173916473985,..."
2,N34694,tv,tvnews,Rosie O'Donnell: Barbara Walters Isn't 'Up to ...,"[-0.042908210307359695, -0.018993420526385307,..."
3,N45794,news,newscrime,Four flight attendants were arrested in Miami'...,"[-0.007930373772978783, -0.0221870094537735, 0..."
4,N18445,sports,football_ncaa,Michigan sends breakup tweet to Notre Dame as ...,"[-0.020153459161520004, -0.044797133654356, 0...."


In [5]:
def get_user_related_content(user):
    user_row = interactions[interactions['User'] == user]
    
    if user_row.empty:
        return [("No data found for user", user)]
    
    user_ids = user_row['ID'].iloc[0].split()
    
    result = []
    
    for user_id in user_ids:
        id_row = news[news['ID'] == user_id]
        
        if not id_row.empty:
            content = id_row['Content'].iloc[0]
            result.append((user_id, content))
    
    user_output = [f'{item[0]}: {item[1]}' for item in result]
    
    return user_output

In [6]:
def clean_id(id_value):
    # Remove index part ("-1" or "-0")
    return id_value.rstrip('01').rstrip('-')

In [7]:
def get_user_related_impressions(user):
    user_row = interactions[interactions['User'] == user]
    
    if user_row.empty:
        return [("No data found for user", user)]
    
    user_ids = user_row['Impressions'].iloc[0].split()
    
    result = []
    
    for user_id in user_ids:
        cleaned_id = clean_id(user_id)
        id_row = news[news['ID'] == cleaned_id]
        
        if not id_row.empty:
            content = id_row['Content'].iloc[0]
            result.append((cleaned_id, content))
            
    impressions_output = [f'{item[0]}: {item[1]}' for item in result]
    
    return impressions_output

In [8]:
def get_news_and_impressions(user_id):
    news_ids = []
    impressions = []
    
    # Extract row corresponding to the user_id
    user_row = interactions[interactions['User'] == user_id]
    
    if not user_row.empty:
        # Extract news IDs and impressions from the row
        news_ids = user_row['ID'].iloc[0].split()
        impressions = user_row['Impressions'].iloc[0].split()
    
    return news_ids, impressions

ChatGPT

In [9]:
openai.api_key = "your-api-key"

In [10]:
# Function to chat with GPT
def chat_with_chatgpt(prompt, model="gpt-3.5-turbo"):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
    )
    message = response['choices'][0]['message']['content'].strip()
    return message

In [11]:
def gpt_task3 (user):
    user_output = get_user_related_content(user)
    impressions_output = get_user_related_impressions(user)
    
    
    user_prompt = (f"The user has interacted with the items from List 1: {user_output}. From the List 2 choose some items, which you could recommend for this user. List 2: {impressions_output}. Output format: a python list with news index (e.g., N12345). Do not explain the reason or include any other words.")
    
    chatbot_response = chat_with_chatgpt(user_prompt)
    
    # Remove leading and trailing whitespaces and newline characters
    cleaned_string = chatbot_response.strip()

    # Use ast.literal_eval to safely evaluate the string as a literal expression
    result_list = ast.literal_eval(cleaned_string)
    
    return (result_list)

In [12]:
def get_random_news(n):
    random_news = random.sample(list(news.itertuples(index=False, name=None)), n)
    return [(news[0], news[3]) for news in random_news]

In [13]:
# n is the number of random user IDs you want to select
n = 2
# Select n random user IDs from the DataFrame
random_user_ids = interactions['User'].sample(n).tolist()
print(random_user_ids)

['U91836', 'U13740']


In [14]:
user_id = "U91836"

In [15]:
gpt_task3(user_id)

['N39317', 'N17059', 'N42977', 'N22407', 'N14592']

In [16]:
#check
check = get_news_and_impressions(user_id)
print(check)

(['N31739', 'N6072', 'N63045', 'N23979', 'N35656', 'N43353', 'N8129', 'N1569', 'N17686', 'N13008', 'N21623', 'N6233', 'N14340', 'N48031', 'N62285', 'N44383', 'N23061', 'N16290', 'N6244', 'N45099', 'N58715', 'N59049', 'N7023', 'N50528', 'N42704', 'N46082', 'N8275', 'N15710', 'N59026', 'N8429', 'N30867', 'N56514', 'N19709', 'N31402', 'N31741', 'N54889', 'N9798', 'N62612', 'N2663', 'N16617', 'N6087', 'N13231', 'N63317', 'N61388', 'N59359', 'N51163', 'N30698', 'N34567', 'N54225', 'N32852', 'N55833', 'N64467', 'N3142', 'N13912', 'N29802', 'N44462', 'N29948', 'N4486', 'N5398', 'N14761', 'N47020', 'N65112', 'N31699', 'N37159', 'N61101', 'N14761', 'N3433', 'N10438', 'N61355', 'N21164', 'N22976', 'N2511', 'N48390', 'N58224', 'N48742', 'N35458', 'N24611', 'N37509', 'N21773', 'N41011', 'N19041', 'N25785'], ['N20678-0', 'N39317-0', 'N58114-0', 'N20495-0', 'N42977-0', 'N22407-0', 'N14592-0', 'N17059-1', 'N33677-0', 'N7821-0', 'N6890-0'])


Evaluation

In [17]:
def create_evaluation_df (user):
    user_row = interactions[interactions['User'] == user]
    user_ids = user_row['Impressions'].iloc[0].split()
    list_1 = user_ids
    list_2 = None
    try:
        list_2 = gpt_task3(user)

    except Exception as e:
        print(f"Error processing user {user}: {e}")
        
    if list_2 is not None:
        # Your code that iterates over list_2 or uses it goes here
        for item in list_2:
            # Process each item in list_2
            pass
    
    #create a DataFrame with the 'ID' column from list_1
    final_df = pd.DataFrame({'ID': [item.split('-')[0] for item in list_1]})

    # Add the 'True' column based on the indexes (1 or 0)
    final_df['True'] = [int(item.split('-')[1]) for item in list_1]

    # Add the 'Predicted' column based on the presence in list_2
    final_df['Predicted'] = [1 if item in list_2 else 0 for item in final_df['ID']]
    
    return final_df

In [18]:
create_evaluation_df(user_id)

,ID,True,Predicted
0,N20678,0,0
1,N39317,0,1
2,N58114,0,0
3,N20495,0,0
4,N42977,0,0
5,N22407,0,1
6,N14592,0,0
7,N17059,1,1
8,N33677,0,1
9,N7821,0,0


In [19]:
def calculate_precision_recall (user):
    df = create_evaluation_df(user)
    # Calculate precision
    precision = precision_score(df['True'], df['Predicted'])

    # Calculate recall
    recall = recall_score(df['True'], df['Predicted'])

    # Calculate weighted precision
    weighted_precision = precision_score(df['True'], df['Predicted'], average='weighted')

    # Calculate weighted recall
    weighted_recall = recall_score(df['True'], df['Predicted'], average='weighted')

    # Print the results
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'Weighted Precision: {weighted_precision}')
    print(f'Weighted Recall: {weighted_recall}')
    

In [20]:
calculate_precision_recall(user_id)

Precision: 0.0
Recall: 0.0
Weighted Precision: 0.7954545454545454
Weighted Recall: 0.6363636363636364
